<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0607bert_classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 認知科学会用 BERT 再検討ファイル

- date: 2022_0607
- author: 浅川伸一

In [ ]:
import platform
hostname = platform.node().split('.')[0]
if hostname == 'Sinope':
    HOME = '/Users/_asakawa'
else:
    HOME = '/Users/asakawa'

import IPython
isColab = 'google.colab' in str(IPython.get_ipython())
if isColab:
    !pip install --upgrade xlrd

    !pip install unidic-lite
    !pip install --upgrade 'fugashi[unidic-lite]'
    #!pip install --upgrade ipadic
    #!pip install --upgrade 'fugashi[ipadic]'
    !pip install --upgrade 'fugashi[unidic]'
    !python -m unidic download
    !pip install --upgrade transformers
    
    !pip install --upgrade termcolor
    !pip install --upgrade jaconv

In [ ]:
import torch
from torch.nn import functional as F
from transformers import BertJapaneseTokenizer
#from transformers import BertForMaskedLM
from transformers import BertForNextSentencePrediction

tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')
model_nsp = BertForNextSentencePrediction.from_pretrained('cl-tohoku/bert-base-japanese')

# GPU が利用可能であれば利用する
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_nsp.to(device)

prompt = "行く川の流れ絶えずして"
#next_sentence = "しかも元の水にあらず。"
next_sentence = "吾輩は猫である。"

encoding = tokenizer.encode_plus(prompt, next_sentence, return_tensors='pt')
outputs = model_nsp(**encoding)[0]
softmax = F.softmax(outputs, dim=1)
# このソフトマックスは，2 つの変数間でなされる。
# すなわち，次の文章であると予測すると 0 番目の確率が高くなり，反対に 1 番目の確率が低くなる
for p in softmax.detach().tolist()[0]:
    print(f'{p:.4f}')

- 次文予測とは，与えられた文に対して，どの程度の文が次の文になるかを予測する作業である。
- この場合 "The child came home from school." が与えられた文であり "He played soccer after school." が次の文であるかどうかを予測しようとしている。
- そのために BERT トークナイザが自動的に文の間に [SEP] トークンを挿入し，2 文の区切りを表し，特定の Bert For Next Sentence Prediction モデルが，その文が次の文であるかどうかの 2 値を予測する。
- Bert は 2 つの値をテンソルで返す。
- 最初の値は 2 番目の文が最初の文の続きであるかどうかを表し，2 番目の値は 2 番目の文がランダムな並びか最初の文の続きでないかを表す。
- 言語モデリングとは異なり BERT の語彙のソフトマックスを計算しようとしているわけではないので，ロジットを取得することはない。
- 我々は，次の文予測のための BERT が返す 2 つの値のソフトマックスを計算しようとしているだけで，どちらの値が最も高い確率値を持っているかを見ることができ，これは第 2 文が第 1 文にとって良い次の文であるかを表すことになる。
- ソフトマックスの値を得たら，それをプリントアウトすることで簡単にテンソルを見ることができる。

Hugging Face は事前に学習されたモデルを持つ課題については，その特定のモデルをダウンロード/インポートする必要があるように設定されている。
この場合，私たちは Bert For Masked Language Modeling モデルをダウンロードしなければなりませんが，トークナイザは前節で述べたように，すべての異なるモデルに対して同じものである。

- マスク言語モデリングは，マスクトークンを任意の位置に挿入し，その位置に入る最適な単語候補を予測することで機能する。
- マスクトークンは，上記のように，入力の希望する位置に連結して挿入すればよい。
- Bert Model for Masked Language Modeling は，その単語を置き換える最適な単語/トークンを語彙として予測する。
- logits は BERT の出力にソフトマックス活性化関数が適用される前の BERT モデルの出力である。
- ロジットを取得するためには，モデルを初期化するときにパラメータで `return_dict = True` を指定する必要があり，そうしないと上記のコードはコンパイルエラーになる。
- BERT Model に入力エンコーディングを渡した後，テンソルを返す 
- `output.logits` を指定するだけで `logits` を得ることができ，この後ようやく `logits` にソフトマックス活性化関数を適用することができる。 
- BERT の出力にソフトマックスを適用することで BERT の語彙の各単語の確率分布を得ることができる。
- より高い確率値を持つ単語はマスクトークンのより良い置換候補となる。
- マスクトークンを置き換えるための BERT の語彙内のすべての単語のソフトマックス値のテンソルを取得するために `torch.where()` を使用して取得したマスクトークンのインデックスを指定することができる。
- この例ではマスクトークンの置換候補単語の上位 10 個を取得しているので (パラメータを適宜調整すれば 10 個以上取得できる)，与えられたテンソル中の上位 $k$ 個の値を取得できる
- `torch.topk()` 関数を使い，その上位 $k$ 個の値を含むテンソルを返している。
- この後はテンソルを繰り返し処理し，文中のマスクトークンを候補トークンに置き換えるだけなので，比較的簡単な処理となる。

In [ ]:
import torch
from torch.nn import functional as F

from transformers import BertJapaneseTokenizer
from transformers import BertForMaskedLM

#model_name = 'cl-tohoku/bert-base-japanese'      # 東北大学乾研による 日本語 BERT 実装
model_name = 'cl-tohoku/bert-base-japanese-v2'

tknz = BertJapaneseTokenizer.from_pretrained(model_name, mecab_dic='unidic')
mlm  = BertForMaskedLM.from_pretrained(model_name, return_dict=True)

# # GPU が利用可能であれば利用する
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
mlm.to(device)

text = "吾輩は猫である。" + tknz.mask_token + "はまだない。"
_input = tknz.encode_plus(text, return_tensors="pt")
mask_index = torch.where(_input["input_ids"][0] == tknz.mask_token_id)
output = mlm(**_input)
logits = output.logits
softmax = F.softmax(logits, dim = -1)
mask_word = softmax[0, mask_index, :]
topN = 10
tokens = torch.topk(mask_word, topN, dim=1)[1][0]
for token in tokens:
    word = tknz.convert_ids_to_tokens(token.detach().item())
    sent_pred = text.replace(tknz.mask_token, word)
    print(sent_pred)


In [ ]:
from transformers import BertForSequenceClassification
classify_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=1742)
print(classify_model.classifier)
# Linear(in_features=768, out_features=1742, bias=True)

# `from_pretrained` 時に `num_labels` を指定する。
# これにより，任意のクラス数の分類器にできる。
# デフォルトでは 2 クラス分類器
# tokenizerと同様キャッシュダウンロードになる。
# なので保存したい場合は下記のようにする。
model_saved_fname = '2022_0607bert_classify_model.pt'
classify_model.save_pretrained(model_saved_fname) # save
classify_model2 = BertForSequenceClassification.from_pretrained(model_saved_fname) # load

In [ ]:
# 2021/Jan 近藤先生からいただいたオノマトペ辞典のデータの読み込み
#'日本語オノマトペ辞典4500より.xls' は著作権の問題があり，公にできません。
# そのため Google Colab での解法，ローカルファイルよりアップロードしてください
import os
import pandas as pd
import jaconv

if isColab:
    from google.colab import files
    uploaded = files.upload()  # ここで `日本語オノマトペ辞典4500より.xls` を指定してアップロードする
    data_dir = '.'
else:
    data_dir = os.path.join(HOME, 'study/2021ccap/notebooks')


onomatopea_excel = '2021-0325日本語オノマトペ辞典4500より.xls'
onmtp2761 = pd.read_excel(os.path.join(data_dir, onomatopea_excel), sheet_name='2761語')

#すべてカタカナ表記にしてデータとして利用する場合
#`日本語オノマトペ辞典4500` はすべてひらがな表記だが，一般にオノマトペはカタカナ表記されることが多いはず
#onomatopea = list(sorted(set([jaconv.hira2kata(o) for o in onmtp2761['オノマトペ']])))

# Mac と Windows の表記の相違を吸収
onomatopea = list(sorted(set([jaconv.normalize(o) for o in onmtp2761['オノマトペ']])))
print(f'データファイル名: {os.path.join(data_dir, onomatopea_excel)}\n',
      f'オノマトペ単語総数: len(onomatopea):{len(onomatopea)}')

# トークナイザ の修正，実際には onomatopea 単語リストを引数に指定して `add_tokens()` を呼び出すだけ
# ただし，語彙数 tknz.vocab は変更されない。追加された語彙，本コードの場合はオノマトペは，
# `tknz.added_tokens_encoder` と `tknz1.added_tokens_decoder` に反映されているためである
num_added = tknz.add_tokens(onomatopea)
print(f'追加されたトークン数:{num_added}/オノマトペ数:{len(onomatopea)}') 
#model_onmt.resize_token_embeddings(len(tknz))
#model_orig.resize_token_embeddings(len(tknz))
classify_model.resize_token_embeddings(len(tknz))
classify_model2.resize_token_embeddings(len(tknz))

print(f' len(tknz):{len(tknz)}\n', 
      f'len(tknz.vocab):{len(tknz.vocab)}\n',  # 一見すると，この数字からオノマトペが追加されていないように見える。
      f'tknz.vocab_size:{tknz.vocab_size}')    # 駄菓子菓子，下で見るように，正しく動作しているように見受けられる

print('# 確認用')
for w in onomatopea[-5:]:
    idx = tknz.convert_tokens_to_ids(w)
    w_ = tknz.convert_ids_to_tokens(idx)
    print(f'単語:{w}(id:{idx}) -> token:{w_}')

In [ ]:
import pandas
import jaconv

if isColab:
    uploaded = files.upload()  # ここで `2022_0531onomatope11.xlsx` をアップロードする
    data_dir = '.'
else:
    data_dir = '..'

df = pandas.read_excel(os.path.join(data_dir, '2022_0531onomatope11.xlsx'))
# 若干の入力ミスと思われる部分を修正した。オリジナルは onomatope11(1).xlsx
bunn = df['bunn'].to_list()
onmtp = df['onomatope'].to_list()
_bunn = [jaconv.normalize(line, 'NFKC') for line in bunn]
_onmtp = [jaconv.normalize(line, 'NFKC') for line in onmtp]
bunn = _bunn
onmtp = _onmtp

ono2sen = {}
sen2ono = {}
for __bunn, __onmtp in zip(bunn, onmtp):
    if not __onmtp in ono2sen:
        ono2sen[__onmtp] = []
    ono2sen[__onmtp].append(__bunn)
    
    if not __bunn in sen2ono:
        sen2ono[__bunn] = []
    sen2ono[__bunn].append(__onmtp)

    
max_cand = 0
for k, v in ono2sen.items():
    n = len(ono2sen[k])
    if n > max_cand:
        max_cand = n
        max_ono = k

print(max_cand, max_ono)
print(ono2sen[max_ono])


max_cand = 0
for k, v in sen2ono.items():
    n = len(sen2ono[k])
    if n > max_cand:
        max_cand = n
        max_sen = k
print(max_cand, max_sen)
print(sen2ono[max_sen])

onomatopea_ids = [tknz.convert_tokens_to_ids(w) for w in onomatopea]
onomatopea == [tknz.convert_ids_to_tokens(tknz.convert_tokens_to_ids(w)) for w in onomatopea]
print(len(onomatopea_ids))